<a href="https://www.kaggle.com/code/parhamgousheh/ovarian-cancer?scriptVersionId=146290917" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Preprocessing

In [1]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing

pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir /kaggle/working/logs --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

--2023-10-12 15:56:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  5.92MB/s    in 2.2s    

2023-10-12 15:56:30 (5.92 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [2]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
https://ba53-104-199-163-3.ngrok.io


In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2
import random
import os
import glob
import torch
import torch.nn as nn
import torchmetrics


from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint


/opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
/opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
/opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /opt/conda/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
TensorBoard 2.12.3 at http://0.0.0.0:6006/ (Press CTRL+C to quit)


In [4]:
pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv').shape

(538, 5)

In [5]:
pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv').head()

,image_id,label,image_width,image_height,is_tma
0,4,HGSC,23785,20008,False
1,66,LGSC,48871,48195,False
2,91,HGSC,3388,3388,True
3,281,LGSC,42309,15545,False
4,286,EC,37204,30020,False


In [6]:
df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
print("Unique Labels and Counts:")
print(df['label'].value_counts())

Unique Labels and Counts:
label
HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: count, dtype: int64


In [7]:
# plt.figure(figsize=(16,24))
# path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
# j=1
# for img, lb in zip(train_df_no_tma['image_id_path'][:24],train_df_no_tma['label'][:24]):
#     plt.subplot(6,4,j)
#     path = os.path.join("/kaggle/input/UBC-OCEAN/train_thumbnails/",img)
#     image = plt.imread(path)
#     image = plt.imshow(image)
#     plt.title(f"Label:{lb}")
#     j+=1

In [8]:
# plt.figure(figsize=(16,28))
# path = "/kaggle/input/UBC-OCEAN/train_images"
# j=1
# for img, lb in zip(train_df_tma['image_id_path'],train_df_tma['label']):
#     plt.subplot(7,4,j)
#     path = os.path.join("/kaggle/input/UBC-OCEAN/train_images",img)
#     image = plt.imread(path)
#     image = plt.imshow(image)
#     plt.title(f"Label:{lb}")
#     j+=1

In [9]:
# Adding image path
df['image_id_path'] = np.where(df['is_tma'], 
                               df['image_id'].astype(str) + ".png", 
                               df['image_id'].astype(str) + "_thumbnail.png")


In [10]:
label_mapping = {'CC': 0, 'EC': 1, 'HGSC': 2, 'LGSC': 3, 'MC': 4}
df['int_label'] = df['label'].map(label_mapping)

In [11]:
# Split dataset into training and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
# Differentiate between TMA and non-TMA
train_df_no_tma = train_df[train_df['is_tma']==False]
train_df_tma = train_df[train_df['is_tma']==True]

val_df_no_tma = val_df[val_df['is_tma']==False]
val_df_tma = val_df[val_df['is_tma']==True]

In [13]:
# Create folders for train and validation images
train_folder = "/kaggle/working/train"
val_folder = "/kaggle/working/val"

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)


In [14]:
# Initialize sums for mean and std calculation
sums = np.zeros(3)
sums_squared = np.zeros(3)
normalizer = 0

# Paths
tumbnail_path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
train_path = "/kaggle/input/UBC-OCEAN/train_images"

# Function to process each dataframe
def process_dataframe(df, folder_name, is_tma=False):
    global sums, sums_squared, normalizer  # Declare as global to modify
    labels_and_paths = []
    source_path = train_path if is_tma else tumbnail_path
    
    for idx, row in df.iterrows():
        img_path = os.path.join(source_path, row['image_id_path'])
        img = cv2.imread(img_path)
        
        # Convert to float and normalize
        img = img.astype(np.float32) / 255.0
        
        # Resize image to fit VGG16 input size
        img_resized = cv2.resize(img, (224, 224))

        if folder_name == 'train':
            # Update sums for mean and std calculation
            for i in range(3):  # Assuming 3 channels: R, G, B
                sums[i] += np.sum(img_resized[:, :, i])
                sums_squared[i] += np.sum(np.square(img_resized[:, :, i]))
            normalizer += img_resized[:, :, 0].size  # size of one channel

        save_path = f"/kaggle/working/{folder_name}/{row['image_id_path']}"
        cv2.imwrite(save_path, img_resized * 255)
        
        labels_and_paths.append({'path': save_path, 'label': row['int_label']})
    
    return pd.DataFrame(labels_and_paths)


In [15]:
# Process and save train and validation images
train_no_tma_labels_df = process_dataframe(train_df_no_tma, 'train')
train_tma_labels_df = process_dataframe(train_df_tma, 'train', is_tma=True)
val_no_tma_labels_df = process_dataframe(val_df_no_tma, 'val')
val_tma_labels_df = process_dataframe(val_df_tma, 'val', is_tma=True)

In [16]:
# Combine the dataframes
train_labels_df = pd.concat([train_no_tma_labels_df, train_tma_labels_df])
val_labels_df = pd.concat([val_no_tma_labels_df, val_tma_labels_df])

In [17]:
# Calculate mean and std
mean = sums / normalizer
std = np.sqrt((sums_squared / normalizer) - np.square(mean))

In [18]:
# Save labels and paths
train_labels_df.to_csv('/kaggle/working/train_labels_and_paths.csv', index=False)
val_labels_df.to_csv('/kaggle/working/val_labels_and_paths.csv', index=False)

In [19]:
print(mean)
print(std)

[0.48814646 0.42956637 0.49082902]
[0.41433804 0.37498681 0.41807137]


# Augmenting

In [20]:
class OvarianCancerDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with image paths and labels.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.dataframe = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  # Assuming that image path is in the first column of the dataframe
        image = Image.open(img_path).convert("RGB")
        label = self.dataframe.iloc[idx, 1]  # Assuming that label is in the second column of the dataframe

        if self.transform:
            image = self.transform(image)

        return image, label


In [21]:
# Define transformations for training and validation datasets
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-10, 10)),  # Rotate between -10 and 10 degrees
    transforms.ToTensor(),
    transforms.Normalize([0.488, 0.429, 0.49], [0.414, 0.374, 0.418])  # Use your own calculated mean and std
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.488, 0.429, 0.49], [0.414, 0.374, 0.418])  # Use your own calculated mean and std
])


In [22]:
# Initialize datasets with different transforms
train_dataset = OvarianCancerDataset(csv_file='/kaggle/working/train_labels_and_paths.csv', transform=train_transform)
val_dataset = OvarianCancerDataset(csv_file='/kaggle/working/val_labels_and_paths.csv', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Model Training

In [23]:
class UnifiedOvarianCancerModel(pl.LightningModule):

    def __init__(self):
        super(UnifiedOvarianCancerModel, self).__init__()

        # Load pre-trained ResNet18 model + higher level features
        self.model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)

        self.learning_rate = 1e-4
        self.num_classes = 5
        self.model.fc = torch.nn.Linear(in_features=512, out_features=self.num_classes, bias=True)

        # Loss function and accuracy metric
        self.criterion = nn.CrossEntropyLoss()
        
        self.train_acc = torchmetrics.Accuracy(num_classes=self.num_classes, task='multiclass')
        self.val_acc = torchmetrics.Accuracy(num_classes=self.num_classes, task='multiclass')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss)
        self.log('train_acc_step', self.train_acc(y_pred, y), on_step=True, on_epoch=False)
        return loss

    def on_training_epoch_end(self):
        self.log('train_acc_epoch', self.train_acc.compute())

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss)
        self.log('val_acc_step', self.val_acc(y_pred, y), on_step=True, on_epoch=False)

    def on_validation_epoch_end(self):
        self.log('val_acc_epoch', self.val_acc.compute())

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [24]:
model = UnifiedOvarianCancerModel()

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=10,
    mode='max',
)

# Initialize the logger
logger = TensorBoardLogger(save_dir='/kaggle/working/logs')

# Initialize the trainer
trainer = Trainer(logger=logger, log_every_n_steps=1, callbacks=checkpoint_callback, max_epochs=10)

# Fit the model
trainer.fit(model, train_loader, val_loader)

Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 211MB/s]


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [25]:
import zipfile
import os

# Initialize zipfile
with zipfile.ZipFile('/kaggle/working/my_archive.zip', 'w') as zipf:
    # Specify the files and folders to zip
    paths_to_zip = [
        '/kaggle/working/logs',
        '/kaggle/working/val',
        '/kaggle/working/train',
        '/kaggle/working/val_labels_and_paths.csv',
        '/kaggle/working/train_labels_and_paths.csv'
    ]
    
    for path in paths_to_zip:
        if os.path.isfile(path):
            # If it's a file, just write it to the zip
            zipf.write(path, os.path.basename(path))
        else:
            # If it's a folder, write all its contents to the zip
            for root, _, files in os.walk(path):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), '/kaggle/working'))
